In [11]:
import numpy as np
import matplotlib.pyplot as plt
import math

def metaF(x, y, M, a, b, mu): #For the FW method you will notice we only require access to the gradient
    #this is R(t), and can be considered step 1 of constructing the LMO
    part1 = mu * (x - a) - np.dot(M, (y - b))
    part2 = np.dot(M.T, (x - a)) + mu * (y - b)

    return np.concatenate([part1, part2])#this is the gradient


def F(x, y, M, a, b, mu):#this just makes it so we can call F on its own
    #they do this in the paper, so we are doing it here
    return metaF(x, y, M, a, b, mu)

def S(F): #this is the FW corner function
    #solves the solution argmin z.T(metaF)
    #was confusing to me, but given the fact that the X and Y are [0,1]^d matrices
    #what we have is Well dxL = x. And dyL = y from xTy
    #[LMOx = -0.5(sign(x) - 1),
    #LMOy = -0.5(sign(y) - 1)]

    return -(np.sign(F + np.finfo(float).eps) - 1) * 0.5


def away_step(Fz, S_t, I_active):
    #grad (numpy.ndarray): The gradient vector.
    #S (numpy.ndarray): The matrix of all possible atoms.
    #I_active (list): The indices of active atoms.
    S = np.array(S_t)
    print(S)
    s = np.dot(Fz.T, S[:, I_active])
    # Find the index of the maximum value in s
    id_max = np.argmax(s)
    # Return the corresponding index from I_active
    return I_active[id_max]

def SP_AFW(Kmax, mu, M, cst, dim, a, b, away, adaptive, tau):
    samples = {
        "gaps" : [],
        "iterations" :[]
    }
    safety = 0

    # Define the problem constants
    delta = min(np.linalg.norm(np.minimum(1 - a, a), 2), np.linalg.norm(np.minimum(1 - b, b), 2))
    #here we choose min(a or 1-a). Norm of that, do that for a and b, and min of that
    #this is *root(min(np.minimum(a))) so its the border distance, square and square rooted (i.e norm), the minimum of both is the delta
    #this is constant
    Pwidth = 1 / np.sqrt(dim)#this is d, dim, just the general length of X and Y
    diam = np.sqrt(dim)
    C_L = mu * diam ** 2
    mu_A = mu * Pwidth ** 2
    mu_int = mu * delta ** 2
    M_L = np.sqrt(2 / mu) * np.linalg.norm(M) * diam


    #tau = 1 - M_L / mu_int

    # Determine the correct C based on the adaptive strategy
    #if adaptive == 4:
    #    C = 2 * mu * diam ** 2 + np.linalg.norm(M) ** 2 * (diam ** 2 / mu + diam ** 2 / mu)
    #else:
    C = 2 * mu * diam ** 2 / tau

    gamma_coef = 1 / C if adaptive > 0 else None

    # Initialize variables
    x_0 = np.concatenate([np.ones(dim), np.zeros(dim)])
    S_0 = x_0
    alpha_0 = 1
    iter = 1
    tol = 1e-6
    G = []
    I_active = np.array([0])
    S_t = np.array([x_0]).T
    mapping = {}
    mapping[hash(tuple(S_t[:, 0]))] = 0

    z = x_0
    S_t = S_0
    alpha_t = alpha_0
    gap_best = np.inf

    for k in range(Kmax):
        Fz = F(z[:dim], z[dim:], M, a, b, mu)
        Sz = S(Fz)
        s_FW = Sz
        grad = Fz
        #s_FW = np.mean(s_FW, axis=1)  # Correcting to (60,)
        #print(s_FW)
        #print(z)
        d_FW = s_FW - z

        gap = -np.dot(d_FW, grad)
        #print(gap)
        #print(gap_best)
        gap_best = min(gap, gap_best)
        G.append(gap_best)

        if gap < tol:
            print(f'Converged at iteration {iter}!')
            break


        if away and adaptive != 3:
            # away direction search:
            if not I_active.size:
                print(f'Error: empty support set at step (iteration={iter})')
            else:
                id_A = away_step(Fz, S_t, I_active)
                v_A = S_t[:, id_A]
                d_A = z - v_A
                alpha_max = alpha_t[id_A]

                # construct direction (between towards and away):
                if -gap <= np.dot(d_A, grad):
                    is_away = False  # this is a pure FW step
                    d = d_FW
                    max_step = 1
                else:
                    is_away = True  # this is an away step
                    number_away += 1
                    d = d_A
                    max_step = alpha_max / (1 - alpha_max)

                gap_AFW = -np.dot(Fz, d)
                if safety:
                    gamma_max = 2 / (k + 2)
                else:
                    gamma_max = 1
                gamma = gamma_coef * gap_AFW
                gamma_k = min(gamma, gamma_max, max_step)
                if adaptive == 3:
                    gamma_k = min(2 / (2 + iter), max_step)
                step = gamma_k

                # Update steps and active set:
                if is_away:
                    # Away step:
                    alpha_t *= (1 + step)
                    if abs(step - max_step) < 10 * np.finfo(float).eps:
                        # Drop step:
                        number_drop += 1
                        alpha_t[id_A] = 0
                        I_active = I_active[I_active != id_A]  # remove from active set
                    else:
                        alpha_t[id_A] -= step
                else:
                    # FW step:
                    alpha_t *= (1 - step)
                    h = hash(tuple(2 * Sz - 1))
                    if h not in mapping:
                        # Add new vertex in S_t:
                        max_index = S_t.shape[1]
                        mapping[h] = max_index
                        S_t = np.hstack((S_t, Sz.reshape(-1, 1)))
                        id_FW = max_index
                        alpha_t = np.append(alpha_t, step)
                        I_active = np.append(I_active, id_FW)
                    else:
                        id_FW = mapping[h]
                        if alpha_t[id_FW] < np.finfo(float).eps:
                            # Track as active:
                            I_active = np.append(I_active, id_FW)
                        alpha_t[id_FW] += step
                    # Exceptional case: stepsize of 1, this collapses the active set!
                    if step > 1 - np.finfo(float).eps:
                        I_active = np.array([id_FW])
        elif adaptive == 3:
            step = 2 / (2 + iter)
            d = d_FW
        else:
            step = min(1, gamma_coef * gap)
            d = d_FW

        # Update position
        z += step * d
        iter += 1
        samples["gaps"].append(math.log10(gap))
        samples["iterations"].append(k)

    return np.array(G), iter, tau


import numpy as np
import matplotlib.pyplot as plt

# Set seed for reproducibility
np.random.seed(2)

# Initialize variables
fEvals = []
fVals = []
Kappas = []
strongs = np.array([50, 60, 100, 300])  # Different values of strong convexity
taus = [.09,.09,.19,.47]
cst = 0.1
dim = 30
a = np.concatenate((-0.5 * np.ones(dim // 2), np.random.rand(dim // 2)))
b = np.concatenate((-0.5 * np.ones(dim // 2), np.random.rand(dim // 2))) #problem P definition of saddle point belonging to the polytope
print(a)
M = cst * (2 * np.random.rand(dim, dim) - 1)
Kmax = 40000
away = 1
adaptive = 3

# First run with adaptive = 3
strong = strongs[0]
G, niter, kappa = SP_AFW(Kmax, strong, M, cst, dim, a, b, away, adaptive, taus[0])
fEvals.append(np.arange(1, niter) / 1e4)
fVals.append(G[:niter-1])
Kappas.append(f'$\\nu = {kappa:.1f} \\gamma = \\frac{{2}}{{2+k(t)}}$')

# Additional runs with different strong values
for i in range(len(strongs)):
    adaptive = 1
    strong = strongs[i]
    G, niter, kappa = SP_AFW(Kmax, strong, M, cst, dim, a, b, away, adaptive, taus[i])
    fEvals.append(np.arange(1, niter) / 1e4)
    fVals.append(G[:niter-1])
    if i == 0:
        Kappas.append(f'$\\nu = {kappa:.1f} \\gamma$ adaptive')
    else:
        Kappas.append(f'$\\nu = {kappa:.2f}$ $\\gamma$ adaptive')

# Plotting options
colors = ['c', 'b', 'g', 'gold', 'r']
line_styles = ['--', '--', '-', '-', '-']
markers = ['s', 'o', 'd', 'p', 'd']
marker_spacing = [4000, 1500, 4000, 1000, 4000, 500, 4000, 800, 4000, 200, 4000, 500]
line_size = 8
legend_font_size = 14

plt.figure(figsize=(10, 6))
for i, (evals, vals) in enumerate(zip(fEvals, fVals)):
    plt.plot(evals, vals, color=colors[i], linestyle=line_styles[i], marker=markers[i],
             markevery=int(marker_spacing[i]), label=Kappas[i], linewidth=line_size)

plt.yscale('log')
plt.legend(loc='upper right', fontsize=legend_font_size)
plt.xlabel('Iteration ($\\times 10^4$)')
plt.ylabel('Duality Gap')
plt.xlim([0, Kmax / 1e4])
plt.ylim([1e-3, 0.2])
plt.grid(True)
plt.show()


[-0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5         0.4359949   0.02592623  0.54966248
  0.43532239  0.4203678   0.33033482  0.20464863  0.61927097  0.29965467
  0.26682728  0.62113383  0.52914209  0.13457995  0.51357812  0.18443987]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed